In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [46]:
spark = SparkSession.builder.appName("sales").getOrCreate()

path = "/content/drive/MyDrive/Colab Notebooks/newDataSets/Sales Data.csv"

df = spark.read.csv(path,header = True, inferSchema = True)

df.show()

+---+--------+--------------------+----------------+----------+----------------+--------------------+-----+------+--------------+----+
|_c0|Order ID|             Product|Quantity Ordered|Price Each|      Order Date|    Purchase Address|Month| Sales|          City|Hour|
+---+--------+--------------------+----------------+----------+----------------+--------------------+-----+------+--------------+----+
|  0|  295665|  Macbook Pro Laptop|               1|    1700.0|30-12-2019 00:01|136 Church St, Ne...|   12|1700.0| New York City|   0|
|  1|  295666|  LG Washing Machine|               1|     600.0|29-12-2019 07:03|562 2nd St, New Y...|   12| 600.0| New York City|   7|
|  2|  295667|USB-C Charging Cable|               1|     11.95|12-12-2019 18:21|277 Main St, New ...|   12| 11.95| New York City|  18|
|  3|  295668|    27in FHD Monitor|               1|    149.99|22-12-2019 15:13|410 6th St, San F...|   12|149.99| San Francisco|  15|
|  4|  295669|USB-C Charging Cable|               1|   

In [47]:
cols_with_missing = [
    F.col for F.col in df.columns if df.filter(df[F.col].isNull()).count() > 0
]
print(cols_with_missing)

# df.filter(F.col("Order ID").isNull()).show()
# df.filter(F.col("Product").isNull()).show()
# df.filter(F.col("Price Each").isNull()).show()
# df.filter(F.col("Order Date").isNull()).show()
# df.filter(F.col("Purchase Address").isNull()).show()
# df.filter(F.col("Month").isNull()).show()
# df.filter(F.col("Sales").isNull()).show()
# df.filter(F.col("City").isNull()).show()
# df.filter(F.col("City").isNull()).show()

df=df.dropna(subset =["Sales"])
# df.filter(F.col("Sales").isNull()).show()

['Sales']


In [49]:
df = df.dropDuplicates()

In [50]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Order ID: integer (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity Ordered: integer (nullable = true)
 |-- Price Each: double (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Purchase Address: string (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Sales: double (nullable = true)
 |-- City: string (nullable = true)
 |-- Hour: integer (nullable = true)



In [51]:
df.show()

+----+--------+--------------------+----------------+----------+----------------+--------------------+-----+------+--------------+----+
| _c0|Order ID|             Product|Quantity Ordered|Price Each|      Order Date|    Purchase Address|Month| Sales|          City|Hour|
+----+--------+--------------------+----------------+----------+----------------+--------------------+-----+------+--------------+----+
| 297|  295941|     ThinkPad Laptop|               1|    999.99|31-12-2019 16:24|64 Dogwood St, Po...|   12|999.99|      Portland|  16|
| 464|  296105|Lightning Chargin...|               1|     14.95|29-12-2019 17:57|134 Dogwood St, S...|   12| 14.95| San Francisco|  17|
| 532|  296169|              iPhone|               1|     700.0|10-12-2019 22:31|111 Hickory St, S...|   12| 700.0|       Seattle|  22|
| 628|  296263|    Wired Headphones|               1|     11.99|11-12-2019 18:04|229 Pine St, San ...|   12| 11.99| San Francisco|  18|
| 721|  296351|     ThinkPad Laptop|            

In [52]:
from pyspark.sql.functions import col

negative_rows = [
    col for col in df.columns if df.filter(df[col] < 0).count() > 0
]

print(negative_rows)

df = df.filter(
    (df["Sales"] >= 0) &
    (df["Price Each"] >= 0) &
    (df["Quantity Ordered"] >= 0)
)

df.show()

['Hour']
+----+--------+--------------------+----------------+----------+----------------+--------------------+-----+------+--------------+----+
| _c0|Order ID|             Product|Quantity Ordered|Price Each|      Order Date|    Purchase Address|Month| Sales|          City|Hour|
+----+--------+--------------------+----------------+----------+----------------+--------------------+-----+------+--------------+----+
| 297|  295941|     ThinkPad Laptop|               1|    999.99|31-12-2019 16:24|64 Dogwood St, Po...|   12|999.99|      Portland|  16|
| 464|  296105|Lightning Chargin...|               1|     14.95|29-12-2019 17:57|134 Dogwood St, S...|   12| 14.95| San Francisco|  17|
| 532|  296169|              iPhone|               1|     700.0|10-12-2019 22:31|111 Hickory St, S...|   12| 700.0|       Seattle|  22|
| 628|  296263|    Wired Headphones|               1|     11.99|11-12-2019 18:04|229 Pine St, San ...|   12| 11.99| San Francisco|  18|
| 721|  296351|     ThinkPad Laptop|   

In [56]:
df_temp = df.groupBy("Product").agg(F.sum("Sales").alias("total_sales"))

df = df.join(df_temp, on="Product",how="left")

df.show(5)

+--------------------+---+--------+----------------+----------+----------------+--------------------+-----+------+--------------+----+------------------+
|             Product|_c0|Order ID|Quantity Ordered|Price Each|      Order Date|    Purchase Address|Month| Sales|          City|Hour|       total_sales|
+--------------------+---+--------+----------------+----------+----------------+--------------------+-----+------+--------------+----+------------------+
|     ThinkPad Laptop|297|  295941|               1|    999.99|31-12-2019 16:24|64 Dogwood St, Po...|   12|999.99|      Portland|  16| 4129958.699999965|
|Lightning Chargin...|464|  296105|               1|     14.95|29-12-2019 17:57|134 Dogwood St, S...|   12| 14.95| San Francisco|  17| 347094.1500000221|
|              iPhone|532|  296169|               1|     700.0|10-12-2019 22:31|111 Hickory St, S...|   12| 700.0|       Seattle|  22|         4794300.0|
|    Wired Headphones|628|  296263|               1|     11.99|11-12-2019 18

In [ ]:
df = df.join(df_temp,)